In [524]:
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [365]:
def dispatch(ctime, location, driver, name, p1=None, p2=None):
        #d = retrieveDriver(driver, es)
        
        p = scanPassenger(location, es)
        
        dDoc = {"ctime": ctime, "location": location, "status": "idle"}
        if p:
            doc = {"status": "pickup", "driver": driver, "ctime": ctime}
            updatePassenger(p['id'], doc, es)

            if p1: 
                dDoc['p2'] = p['id']
            else:
                dDoc['p1'] = p['id']
                
            dDoc['status'] = "pickup"
            dDoc['destination'] = p['location']
            dDoc['altdest1'] = p['id']
            

        updateDriver(driver, dDoc, es)
        
        bulk = (1, '{{doc: {}}}'.format(json.dumps(dDoc)))
        return (bulk)

In [ ]:
      'driver': {'type': 'string', "include_in_all": "false"},
        'destination': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'destinationid':{'type': 'string', "include_in_all": "false"},                
        'altdest1': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid1':{'type': 'string', "include_in_all": "false"},                    
        'altdest2': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid2':{'type': 'string', "include_in_all": "false"}, 
        'origin': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'path': {'type': 'string', "include_in_all": "false"}


In [540]:
window = 'now-2h'
location =[40.7589, -73.985]

def retrievePassenger(pID, es):
    p = es.get(index='passenger', doc_type='rolling', id=pID, \
                   ignore=[404, 400])
    return p['_source'] if p['found'] else False


p = retrievePassenger(44, es)
if p: 
    shoulds = []
    for i in [p['destinationid'],p['altdest1id'],p['altdest1id']]:
        shoulds.append({'match': {'destinationid': i}})
        shoulds.append({'match': {'altdest1id': i}}) 
        shoulds.append({'match': {'altdest1id': i}}) 

    destinations = [p['destinationid'], p['altdest1id'], p['altdest2id']]
    geo_query = {"size": 1, 
         "query" : {
          "bool" : {
          "must" : { "term" : { "status" : "arrived" }},
      "must_not" : { "term" : { "id" : p['id'] }},

         "filter": {
        "geo_distance": {
            "distance": '3km',
       "distance_type": "plane", 
            "location": location }},

        "should" : shoulds,
      "minimum_should_match" : 1,
                     "boost" : 1.0
            }},
           "sort": [{
  "_geo_distance": {
       "location": location,
          "order": "asc",
           "unit": "km", 
  "distance_type": "plane" 
          }}],
                }
    
res = es.search(index='passenger', doc_type='rolling', body=geo_query, ignore=[404, 400])



In [542]:
print res['hits']['hits'][0]["_source"] if res['hits']['hits'] else False


{u'status': u'arrived', u'city': u'NYC', u'altdest2id': u'9/11 Memorial & Museum', u'altdest2': [40.7114, -74.0125], u'altdest1': [40.7794, -73.9632], u'altdest1id': u'The Metropolitan Museum of Art', u'destination': [40.7588, -73.9851], u'driver': 25, u'ctime': u'2016-10-11T05:31:24.189421Z', u'destinationid': u'Times Square', u'location': [40.7589, -73.985], u'ptime': 0, u'path': [[40.7887, -73.984], [40.8078, -73.9686], [40.7833, -73.9723], [40.7988, -73.9742], [40.7887, -73.984], [40.7887, -73.984], [40.7588, -73.9851], [40.7588, -73.9851], [40.7588, -73.9851]], u'id': 77, u'name': u'passenger_77'}


In [480]:
res = retrieveDriver(690990909, es)

In [481]:
res

{u'_id': u'690990909',
 u'_index': u'driver',
 u'_type': u'rolling',
 u'found': False}